In [ ]:
!pip install llama_index
!pip install langchain


from llama_index import ( 
    SimpleDirectoryReader,
    GPTListIndex,
    readers,
    GPTSimpleVectorIndex, 
    LLMPredictor,
    PromptHelper
)
from langchain import OpenAI

import sys
from google.colab import drive
import os
drive.mount("/content/gdrive")

os.environ["OPENAI_API_KEY"] = ""

def construct_index(directory_path):
  # max input size
  max_input_size = 4096

  # set number of output tokens
  num_outputs = 256

  # set maxmium chunk overlap
  max_chunk_overlap = 20

  # set chunk size
  chunk_size_limit = 600

  # define LLM
  llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-ada-001", max_retries=num_outputs))
  prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
  
  documents = SimpleDirectoryReader(directory_path).load_data()

  # build index
  index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

  # get response from query
  index.save_to_disk("index.json")

  return index


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

book_path = input("Path to the book in Drive-")
construct_index(book_path)

Path to the book in Drive-/content/gdrive/MyDrive/meditation/g.txt


In [ ]:
def ask_marcus():
  index = GPTSimpleVectorIndex.load_from_disk("index.json")

  while True:
    query = input("Ask Marcus")
    response = index.query(query, response_mode="compact", verbose=True)
    print("\n\n Marcus says: " + response.response )